In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# Equations of Motion and Numerical Integration

## Equations of Motion

Consider two point masses, $ \mathnormal{m_1} $ and $ \mathnormal{m_2} $
  
![two-body problem inertial frame](images/tbp_inertial_pointmasses.png)

$ \mathcal{F}_I $ is an inertial frame.  An inertial frame is a coordinate frame where Newton's Laws apply.

$ \vec{\mathbf{r}}_1 $ is the position vector of mass $ \mathnormal{m_1} $. 

$ \vec{\mathbf{r}}_2 $ is the position vector of mass $ \mathnormal{m_2} $. 

$ \vec{\mathbf{r}}_{21} $ is the position vector of mass $ \mathnormal{m_2} $ with relative to $ \mathnormal{m_1} $.
\begin{equation}
\vec{\mathbf{r}}_{21} = \vec{\mathbf{r}}_{2} - \vec{\mathbf{r}}_{1}
\end{equation}


The force exerted by each mass can be determined by Newton's law of gravitation.

The force $ \mathnormal{m_2} $ exerts on $ \mathnormal{m_1} $:
\begin{equation}
\vec{\mathbf{F}}_{12} = \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

The force $ \mathnormal{m_1} $ exerts on $ \mathnormal{m_2} $:
\begin{equation}
\vec{\mathbf{F}}_{21} = - \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

$\mathnormal{G}$ is Newton's universdal gravitational constant.  
\begin{equation}
\mathnormal{G} = 6.67 \times 10^{-11}  \mathrm{m}^3 / (\mathrm{kg} \cdot \mathrm{s}^2)
\end{equation}

Applying Newton's second law:

$ \mathnormal{m_1} $:  
\begin{equation}
\mathnormal{m_1} \ddot{\vec{\mathbf{r}}}_{1} = \vec{\mathbf{F}}_{12} = \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

$ \mathnormal{m_2} $:
\begin{equation}
\mathnormal{m_2} \ddot{\vec{\mathbf{r}}}_{2} = \vec{\mathbf{F}}_{21} = - \frac{\mathnormal{G}\mathnormal{m_1}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

Yields:

\begin{equation}
\ddot{\vec{\mathbf{r}}}_{1} = \frac{\mathnormal{G}\mathnormal{m_2}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}
\begin{equation}
\ddot{\vec{\mathbf{r}}}_{2} = -\frac{\mathnormal{G}\mathnormal{m_1}}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

Taking the difference:
\begin{equation}
\ddot{\vec{\mathbf{r}}}_{21} = \ddot{\vec{\mathbf{r}}}_{2} - \ddot{\vec{\mathbf{r}}}_{1} = - \frac{\mathnormal{G}(\mathnormal{m_1}+\mathnormal{m_2})}{|\vec{\mathbf{r}}_{21}|^3} \vec{\mathbf{r}}_{21}
\end{equation}

If $ \mathnormal{m_2} \ll \mathnormal{m_1}$, such as a satellite orbiting the Earth, we can assume $ \mathnormal{m_1} + \mathnormal{m_2} \approx \mathnormal{m_1} $ and define $ \mu = \mathnormal{G}\mathnormal{m_1} $, where $ \mathnormal{m_1} $ is the primary body and $ \mathnormal{m_2} $ is the secondary body.

After dropping the subscripts, the relative equations of motion are:
\begin{equation}
\ddot{\vec{\mathbf{r}}} = - \frac{\mu}{|\vec{\mathbf{r}}|^3} \vec{\mathbf{r}}
\end{equation}


### Standard Gravitational Parameter

Earth, $ \oplus $:  
\begin{equation}
\mu_{\oplus} = 3.986 \times 10^{14}  \mathrm{m}^3 / \mathrm{s}^2 
\end{equation}

Sun, $ \odot $:  
\begin{equation}
\mu_{\odot} = 1.327 \times 10^{20}  \mathrm{m}^3 / \mathrm{s}^2 
\end{equation}


In [ ]:
try:
    from modsim import *
except ImportError:
    !pip install modsimpy
    from modsim import *